# Tyche Example with Simple PV Model

## Set up.

### One only needs to execute the following line once, in order to make sure recent enough packages are installed.

In [3]:
#pip install numpy>=1.17.2 pandas>=0.25.1

### Import packages.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import re                as re
import scipy.stats       as st
import seaborn           as sb

# The `tyche` package is located at <https://github.com/NREL/portfolio/tree/master/production-function/framework/src/tyche/>.
import tyche             as ty

from copy import deepcopy

## Load data.

### The data are stored in a set of tab-separated value files in a folder.

In [6]:
designs = ty.Designs(".")

In [7]:
investments = ty.Investments(".")

### Compile the production and metric functions for each technology in the dataset.

In [8]:
designs.compile()

## Examine the data.

### The `functions` table specifies where the Python code for each technology resides.

In [9]:
designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
Simple pv,numpy,utility_pv,capital_cost,fixed_cost,production,metrics,


Right now, only the style `numpy` is supported.

### The `indices` table defines the subscripts for variables.

In [10]:
designs.indices

Offset               Description  \
Technology Type    Index                                                  
Simple pv  Capital Other Capital Cost       0        Other Capital Cost   
           Input   Solar Radiation          0           Solar Radiation   
           Metric  GHG                      1  Greenhouse gas emissions   
                   LCOE                     0                      Cost   
           Output  Electricity              0               Electricity   

                                                                                   Notes  
Technology Type    Index                                                                  
Simple pv  Capital Other Capital Cost  Placeholder in case other capital costs are ne...  
           Input   Solar Radiation                                                        
           Metric  GHG                                                                    
                   LCOE                                                                   
           Output  Electricity

### The `designs` table contains the cost, input, efficiency, and price data for a scenario.

In [11]:
designs.designs

Value  \
Technology Scenario Variable          Index                       
Simple pv  Base PV  Input             Solar Radiation       5.5   
                    Input efficiency  Solar Radiation     0.152   
                    Input price       Solar Radiation         0   
                    Lifetime          Other Capital Cost     20   
                    Output efficiency Electricity             1   
                    Output price      Electricity         0.092   
                    Scale             NaN                  0.05   

                                                                   Units  \
Technology Scenario Variable          Index                                
Simple pv  Base PV  Input             Solar Radiation         kWh/m2/day   
                    Input efficiency  Solar Radiation                  1   
                    Input price       Solar Radiation     USD/kWh/m2/day   
                    Lifetime          Other Capital Cost              yr   
                    Output efficiency Electricity                      1   
                    Output price      Electricity                USD/kWh   
                    Scale             NaN                      module/yr   

                                                                                                      Notes  
Technology Scenario Variable          Index                                                                  
Simple pv  Base PV  Input             Solar Radiation                                                        
                    Input efficiency  Solar Radiation                             From Kavlak et al. (2018)  
                    Input price       Solar Radiation                                                        
                    Lifetime          Other Capital Cost  Assumed, Kavlak et al. (2019) do not provide a...  
                    Output efficiency Electricity                         No output inverter losses assumed  
                    Output price      Electricity                     Average commercial rate in Denver, CO  
                    Scale             NaN                 Inverse of lifetime. Constant needed to leveli...

### The `parameters` table contains additional techno-economic parameters for each technology.

In [12]:
designs.parameters

Offset     Value        Units  \
Technology Scenario Parameter                                                   
Simple pv  Base PV  Cells per module                 0        72  cell/module   
                    GHG factor for electricity      13       400    gCO2e/kWh   
                    Module area utilization         12       0.9     unitless   
                    Non-silicon materials cost       6  0.009433   $/cm2/cell   
                    Plant size                       8      1000        MW/yr   
                    Polysilicon price                4        26         $/kg   
                    Production yield                11      0.95     unitless   
                    Reference plant cost             7    1.5513       $/cell   
                    Reference plant size             9      1000        MW/yr   
                    Scaling factor                  10      0.27     unitless   
                    Silicon utilization              5      0.45     unitless   
                    Wafer area                       1       243          cm2   
                    Wafer density                    3      2.33        g/cm3   
                    Wafer thickness                  2       180           um   

                                                                                            Notes  
Technology Scenario Parameter                                                                      
Simple pv  Base PV  Cells per module                                    From Kavlak et al. (2018)  
                    GHG factor for electricity                    Rough approximation for US Grid  
                    Module area utilization                             From Kavlak et al. (2018)  
                    Non-silicon materials cost  Calculated based on data from Kavlak et al. (2...  
                    Plant size                  From Kavlak et al. (2018). Equivalent to 3.35E...  
                    Polysilicon price                            2015$. From Kavlak et al. (2018)  
                    Production yield            Production waste parameter. Include as an outp...  
                    Reference plant cost        Calculated based on data from Kavlak et al. (2...  
                    Reference plant size        From Kavlak et al. (2018). Equivalent to 3.35E...  
                    Scaling factor                                      From Kavlak et al. (2018)  
                    Silicon utilization                                 From Kavlak et al. (2018)  
                    Wafer area                                          From Kavlak et al. (2018)  
                    Wafer density                                       From Kavlak et al. (2018)  
                    Wafer thickness                                     From Kavlak et al. (2018)

### The `results` table specifies the units of measure for results of computations.

In [13]:
designs.results

Units Notes
Technology Variable Index                          
Simple pv  Cost     Cost           USD/module      
           Metric   GHG          gCO2e/module      
                    LCOE              USD/kWh      
           Output   Electricity    kWh/module

### The `tranches` table specifies multually exclusive possibilities for investments: only one `Tranch` may be selected for each `Cateogry`.

In [14]:
investments.tranches

Amount Notes
Category Tranche       Scenario                                
PV R&D   High PV R&D   Fast Progress on PV      1000000.0      
         Low PV R&D    Slow Progress on PV      3000000.0      
         Medium PV R&D Moderate Progress on PV  4500000.0      
         No PV R&D     Base PV                  1500000.0

### The `investments` table bundles a consistent set of tranches (one per category) into an overall investment.

In [15]:
investments.investments

,,,Notes
Investment,Category,Tranche,
No R&D Spending,PV R&D,No PV R&D,


## Evaluate the scenarios in the dataset.

In [16]:
scenario_results = designs.evaluate_scenarios()

Evaluating Simple pv


C:\Users\rhanes\AppData\Local\Continuum\anaconda3\envs\tyche\lib\site-packages\numpy\core\fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [17]:
scenario_results.xs(1, level="Sample", drop_level=False)

Value         Units
Technology Scenario Sample Variable Index                                  
Simple pv  Base PV  1      Cost     Cost        -7.177702e+02    USD/module
                           Metric   GHG          4.508260e+06  gCO2e/module
                                    LCOE        -6.368489e-02       USD/kWh
                           Output   Electricity  1.127065e+04    kWh/module

### Save results.

In [18]:
#scenario_results.to_csv("output/utility_pv/results.csv")